In [1]:
import import_ipynb
from ConsistencyIndexes import *

importing Jupyter notebook from ConsistencyIndexes.ipynb
importing Jupyter notebook from Functions.ipynb


In [2]:
class ParamInvividual:
    def __init__(self,N=2,idx=[CRC,Border_Err],W=[1,1],M="mean"):
        self.NIdx = N
        self.Idx = idx
        self.Weights = W
        self.Mode = M
        self.Score = 0

In [3]:
@numba.jit
def CopyIndv(Inv):
    return ParamInvividual(Inv.NIdx,Inv.Idx,Inv.Weights,Inv.Mode)

In [7]:
@numba.jit
def Generator():
    Indexes = [Abs_Dif,Gray_Dif,MSE,Border_Err,CRC,Entropy_Dif,Freq_Dif,TSNR,TSSIM,OF_EPE,OF_AE]
    Modes = ["mean","log","norm","Z"]
    Nindex = random.sample(range(0,len(Indexes)),random.randint(2,3))
    Idx = [Indexes[i] for i in Nindex]
    W = [random.uniform(0.1, 1) for _ in range(len(Nindex))]
    M = Modes[random.randint(0,len(Modes)-1)]
    return ParamInvividual(len(Nindex),Idx,W,M)

In [8]:
@numba.jit
def Gen_Population(N=100):
    return [Generator() for _ in range(N)]

In [9]:
@numba.jit
def Evaluation(Individual,Frames,Targets):
    Inc,Metric = InconsistentRegion(Frames,Mix_Metrics,Individual.Mode,Individual.Idx,Individual.Weights)
    Inc = Inc[1:]
    Dif = [abs(Inc[i]-Targets[i]) for i in range(len(Inc))]
    return np.mean(sum(Dif)/len(Dif)) #+ sum([0 if d.all()==np.zeros_like(Targets[0]).all() else 1 for d in Targets])# + Metric)

In [10]:
@numba.jit
def Evaluate_Population(Population,Frames,Targets):
    for i in range(len(Population)):
        print("Pop: ",i+1,'/',len(Population),end='\r')
        Population[i].Score = Evaluation(Population[i],Frames,Targets)

In [11]:
@numba.jit
def Read2Binary(path):
    T = read_images(path)
    T = [cv2.cvtColor(t,cv2.COLOR_RGB2GRAY) for t in T]
    for t in T:
        t[t>=150]=255
        t[t<150]=0
    return T

In [12]:
def Selection(Population,NTop=10):
    Scores = [P.Score for P in Population]
    Dictionary = dict(zip(Population, Scores))
    Sorted = sorted(Dictionary.items(), key=lambda x: x[1])
    return [key for key, value in Sorted[:NTop]]

In [13]:
@numba.jit
def Cross(Indv1,Indv2):
    N = (Indv1.NIdx+Indv2.NIdx)//2
    Dic = dict(zip(Indv1.Idx+Indv2.Idx,Indv1.Weights+Indv2.Weights))
    idx = random.sample(list(Dic.keys()), N)
    W = [Dic[key] for key in idx]
    M = [Indv1.Mode,Indv2.Mode][random.randint(0,1)]
    return ParamInvividual(N,idx,W,M)

In [14]:
@numba.jit
def Mutation(Indv):
    I = CopyIndv(Indv)
    if random.randint(0,1)==0:
        I.Weights = [random.uniform(0.1, 10) for _ in range(Indv.NIdx)]
        return I
    else:
        return Generator()

In [15]:
@numba.jit
def Gen_From_Top(N,Top):
    Population = []
    for n in range(N):
        Op = random.randint(0,1)
        if Op==0:
            I = Cross(Top[random.randint(0,len(Top)-1)],Top[random.randint(0,len(Top)-1)])
        else:
            I = Mutation(Top[random.randint(0,len(Top)-1)])
        Population.append(I)
    return Population

In [16]:
@numba.jit
def Train(Frames,Targets,NPop=5,Generations=3,NTop=3):
    Population = Gen_Population(NPop)
    Evaluate_Population(Population,Frames,Targets)
    for G in range(Generations):
        print("Gen: ",G+1,"/",Generations,'\n')
        Top = Selection(Population,NTop)
        Children = Gen_From_Top(NPop-NTop,Top)
        Evaluate_Population(Children,Frames,Targets)
        Population = Top+Children
    Top = Selection(Population,NTop)
    return Top[0]

In [17]:
# If Mask is zeros add to diff

In [18]:
Frames = read_images("saved_frames")

In [19]:
I,B = AddInc(Frames)

In [20]:
len(I)

69

In [21]:
Best = Train(I[:15],B[:14],20,15,10)

Gen:  1 / 15 



KeyboardInterrupt: 

In [ ]:
Best.Mode,Best.Idx,Best.Weights

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,Best.Mode,Best.Idx,Best.Weights)
ThroughFrames(Img)

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"mean",[CRC,Gray_Dif],[0.6206048025607019, 0.508216164936656])
ThroughFrames(Img)

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"log",[MSE,Entopy_Dif,CRC], [0.6795718573404556, 0.3460195810616382, 0.9312673477407468])
ThroughFrames(Img)

In [ ]:
Img = DrawInconsistancy1(I[:25],Mix_Metrics,"log",[Entopy_Dif,CRC],  [0.7347384902639871, 0.3513275883482687])
ThroughFrames(Img)

('mean',
 [<function ConsistencyIndexes.CRC(image1, image2)>,
  <function ConsistencyIndexes.Gray_Dif(image1, image2)>],
 [0.6206048025607019, 0.508216164936656])

('log',
 [<function ConsistencyIndexes.MSE(image1, image2)>,
  <function ConsistencyIndexes.Entopy_Dif(image1, image2)>,
  <function ConsistencyIndexes.CRC(image1, image2)>],
 [0.6795718573404556, 0.3460195810616382, 0.9312673477407468])

('mean',
 [<function ConsistencyIndexes.Gray_Dif(image1, image2)>,
  <function ConsistencyIndexes.CRC(image1, image2)>],
 [0.22467344683611584, 0.5673037121649521])

('log',
 [<function ConsistencyIndexes.Entopy_Dif(image1, image2)>,
  <function ConsistencyIndexes.CRC(image1, image2)>],
 [0.7347384902639871, 0.3513275883482687])

Seguimiento de error en frames consecutivos<br>
Filtros diferentes de cartoonization<br>
OF en bordes<br>
Fracuencias en Imagenes<br>
Diferencia en imagen real y caricaturizada<br>


In [26]:
@numba.jit
def calculate_optical_flow_difference(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")

    # Apply Canny edge detection
    edges1 = cv2.Canny(img1, 100, 200)
    edges2 = cv2.Canny(img2, 100, 200)
    edges3 = cv2.Canny(img3, 100, 200)

    # Compute optical flow between the first and second images
    flow1 = cv2.calcOpticalFlowFarneback(edges1, edges2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute optical flow between the second and third images
    flow2 = cv2.calcOpticalFlowFarneback(edges2, edges3, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1[..., 0]**2 + flow1[..., 1]**2)
    angle1 = np.arctan2(flow1[..., 1], flow1[..., 0])
    
    magnitude2 = np.sqrt(flow2[..., 0]**2 + flow2[..., 1]**2)
    angle2 = np.arctan2(flow2[..., 1], flow2[..., 0])

    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)

    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero

    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index

    return difference_index